In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import dotenv
from sqlalchemy import create_engine, text

dotenv.load_dotenv()

True

In [2]:
# Start db
def create_conn():
    driver = os.getenv("DB_DRIVER")
    server = os.getenv("DB_SERVER")
    database = os.getenv("DB_NAME")
    trusted_connection = os.getenv("DB_TRUSTED_CONNECTION")

    return create_engine(
        f"mssql+pyodbc://{server}/{database}?trusted_connection={trusted_connection}&driver={driver}"
    )


engine = create_conn()
# Test connection
connection = engine.connect()
res = connection.execute(text("SELECT @@version;")).fetchone()
connection.close()
res[0]

'Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) \n\tOct  8 2022 05:58:25 \n\tCopyright (C) 2022 Microsoft Corporation\n\tDeveloper Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 19045: ) (Hypervisor)\n'

In [3]:
query_campaigns = """
SELECT
c.crm_campagne_id, c.crm_campagne_naam, c.crm_campagne_startdatum, c.crm_campagne_type_campagne, c.crm_campagne_soort_campagne, i.crm_contact_id
FROM campagne c INNER JOIN inschrijving AS i ON c.crm_campagne_id = i.crm_campagne_id WHERE 
c.crm_campagne_status = 'Actief'
"""

query_contacts = """
SELECT 
/*Contact*/
co.crm_contact_id, co.crm_contact_functietitel, crm_contact_voka_medewerker, 

/*Persoon*/
p.crm_persoon_mail_thema_duurzaamheid, p.crm_persoon_mail_thema_financieel_fiscaal, 
p.crm_persoon_mail_thema_innovatie, p.crm_persoon_mail_thema_internationaal_ondernemen, 
p.crm_persoon_mail_thema_mobiliteit, p.crm_persoon_mail_thema_omgeving, p.crm_persoon_mail_thema_sales_marketing_communicatie, p.crm_persoon_mail_thema_strategie_en_algemeen_management, 
p.crm_Persoon_Mail_thema_talent, p.crm_persoon_mail_thema_welzijn, p.crm_persoon_mail_type_bevraging, 
p.crm_persoon_mail_type_communities_en_projecten, p.crm_persoon_mail_type_netwerkevenementen, p.crm_persoon_mail_type_nieuwsbrieven, 
p.crm_persoon_mail_type_opleidingen, p.crm_persoon_mail_type_persberichten_belangrijke_meldingen, p.crm_persoon_marketingcommunicatie,

/*Account*/
a.crm_account_is_voka_entiteit, a.crm_account_ondernemingsaard, a.crm_account_ondernemingstype,
a.crm_account_primaire_activiteit, a.crm_account_id,

/*Financiele data*/
fd.crm_financieledata_toegevoegde_waarde, fd.crm_financieledata_fte, fd.crm_financieledata_aantal_maanden, fd.crm_financieledata_boekjaar,

/*Info & klachten*/
ik.aantal_klachten,

/*Activiteitscode*/
act.crm_activiteitscode_naam,

/*Afspraak betreft account*/
aba.crm_afspraak_betreft_account_thema, aba.crm_afspraak_betreft_account_subthema, aba.crm_afspraak_betreft_account_onderwerp,
aba.crm_afspraak_betreft_account_keyphrases, aba.crm_afspraak_id AS 'crm_afspraak_betreft_account_id',

/*Afspraak betreft contact*/
abc.crm_afspraak_id AS 'crm_afspraak_betreft_contact_id', abc.crm_afspraak_betreft_contactfiche_thema, abc.crm_afspraak_betreft_contactfiche_subthema, abc.crm_afspraak_betreft_contactfiche_onderwerp, abc.crm_afspraak_betreft_contactfiche_keyphrases

FROM contact AS co
INNER JOIN persoon AS p ON co.crm_persoon_id = p.crm_persoon_id
INNER JOIN account AS a ON a.crm_account_id = co.crm_contact_account
INNER JOIN financiele_data AS fd ON a.crm_account_id= fd.crm_account_id
INNER JOIN (
	SELECT COUNT(*) AS 'aantal_klachten', crm_account_id FROM info_en_klachten 
	GROUP BY crm_account_id
) AS ik ON ik.crm_account_id = a.crm_account_id
INNER JOIN account_activiteitscode AS aa ON a.crm_account_id= aa.crm_account_id
INNER JOIN activiteitscode AS act ON act.crm_activiteitscode_id = aa.crm_activiteitscode_id
INNER JOIN afspraak_betreft_account AS aba ON aba.crm_account_id = a.crm_account_id
INNER JOIN afspraak_betreft_contact AS abc ON abc.crm_contact_id = co.crm_contact_id
WHERE 
co.crm_contact_status = 'Actief' 
AND p.crm_persoon_reden_van_status = 'Actief'
AND a.crm_account_status = 'Actief'
AND act.crm_activiteitscode_status = 'Actief'
"""
# TODO: Kernwoorden op 1 rij

In [4]:
df_campaigns = pd.read_sql(query_campaigns, engine)
df_campaigns

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,6BE9AE3C-1B6D-E111-B43A-00505680000A
1,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,B61B99D1-34D0-EB11-8120-001DD8B72B61
2,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,A8A15926-6974-E111-B43A-00505680000A
3,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,48D9A655-5FD7-EC11-A7B5-000D3AACFF7A
4,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BD6E0D2C-446B-E111-B43A-00505680000A
...,...,...,...,...,...,...
49863,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,9DFA0E37-89A5-E611-80DE-001DD8B72B61
49864,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,1A05DA64-4F67-EE11-9AE7-6045BD895FE3
49865,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,A3E9E223-5167-EE11-9AE7-6045BD8952CE
49866,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,1F09859E-4772-ED11-9561-6045BD895CDC


In [5]:
df_contacts = pd.read_sql(query_contacts, engine)
df_contacts

In [ ]:
# Create csv by joining the two dataframes on contact_id

df_o = pd.merge(df_campaigns, df_contacts, left_on="crm_contact_id", right_on="crm_contact_id")
df = df_o.copy()
df

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,...,crm_afspraak_betreft_account_thema,crm_afspraak_betreft_account_subthema,crm_afspraak_betreft_account_onderwerp,crm_afspraak_betreft_account_keyphrases,crm_afspraak_betreft_account_id,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contactfiche_thema,crm_afspraak_betreft_contactfiche_subthema,crm_afspraak_betreft_contactfiche_onderwerp,crm_afspraak_betreft_contactfiche_keyphrases
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2019,"grote administratieve eisen , alimetro specifi...",F1CC57DC-F581-E911-80FF-001DD8B72B62,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
1,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2019,"grote administratieve eisen , alimetro specifi...",F1CC57DC-F581-E911-80FF-001DD8B72B62,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
2,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2019,"grote administratieve eisen , alimetro specifi...",F1CC57DC-F581-E911-80FF-001DD8B72B62,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
3,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"sabine en kathleen , echtgenoot kathleen , sab...",A3183E14-7C2B-E811-80F0-001DD8B72B61,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
4,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"sabine en kathleen , echtgenoot kathleen , sab...",A3183E14-7C2B-E811-80F0-001DD8B72B61,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243917,3BAA0411-71A0-EC11-B400-6045BD93CA86,OV-NW-Voka Vista - Community event 05/2022,2022-05-05 08:00:00,Netwerkevenement,Offline,E0A73760-4714-EB11-8114-001DD8B72B61,Managing Director,0,Ja,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-klantenbezoek 2018,"[NAME] [NAME] , veel problemen , technische pr...",3EF342F7-985E-E811-80F1-001DD8B72B62,C97A9D1D-36A1-ED11-AAD0-6045BD895BFB,Lidmaatschap,Retentie (Lidmaatschap),OV-klantenbezoek 2023,"[NAME] [NAME] [NAME] [NAME] , de meeste duurza..."
243918,3BAA0411-71A0-EC11-B400-6045BD93CA86,OV-NW-Voka Vista - Community event 05/2022,2022-05-05 08:00:00,Netwerkevenement,Offline,E0A73760-4714-EB11-8114-001DD8B72B61,Managing Director,0,Ja,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-klantenbezoek 2018,"[NAME] [NAME] , veel problemen , technische pr...",3EF342F7-985E-E811-80F1-001DD8B72B62,C97A9D1D-36A1-ED11-AAD0-6045BD895BFB,Lidmaatschap,Retentie (Lidmaatscha

In [ ]:
# Filter out the rows where crm_boekjaar != YEAR(crm_campagne_startdatum)
df["crm_campagne_startdatum"] = pd.to_datetime(df["crm_campagne_startdatum"])
df["crm_campagne_startdatum_year"] = df["crm_campagne_startdatum"].dt.year

df = df[df["crm_campagne_startdatum_year"] == df["crm_financieledata_boekjaar"]]
df = df.drop(columns=["crm_campagne_startdatum_year"])
original_len = len(df)
df

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,...,crm_afspraak_betreft_account_thema,crm_afspraak_betreft_account_subthema,crm_afspraak_betreft_account_onderwerp,crm_afspraak_betreft_account_keyphrases,crm_afspraak_betreft_account_id,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contactfiche_thema,crm_afspraak_betreft_contactfiche_subthema,crm_afspraak_betreft_contactfiche_onderwerp,crm_afspraak_betreft_contactfiche_keyphrases
21,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"sabine en kathleen , echtgenoot kathleen , sab...",A3183E14-7C2B-E811-80F0-001DD8B72B61,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
37,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2019,"grote administratieve eisen , alimetro specifi...",F1CC57DC-F581-E911-80FF-001DD8B72B62,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
56,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2019,"grote administratieve eisen , alimetro specifi...",F1CC57DC-F581-E911-80FF-001DD8B72B62,3146948E-6B40-EB11-8116-001DD8B72B61,"Digitalisering, IT & Technologie",Quick Scan,Opvolggesprek Quick Scan Deloitte,"[NAME] [NAME] [NAME] [NAME] , [NAME] [NAME] in..."
75,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"sabine en kathleen , echtgenoot kathleen , sab...",A3183E14-7C2B-E811-80F0-001DD8B72B61,3146948E-6B40-EB11-8116-001DD8B72B61,"Digitalisering, IT & Technologie",Quick Scan,Opvolggesprek Quick Scan Deloitte,"[NAME] [NAME] [NAME] [NAME] , [NAME] [NAME] in..."
169,3806752A-D9E0-E711-80EE-001DD8B72B61,OV-P-Groep 268-PP.Toeleveranciers Retail-Plato...,2018-11-23 06:00:00,Project,Offline,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"sabine en kathleen , echtgenoot kathleen , sab...",A3183E14-7C2B-E811-80F0-001DD8B72B61,5B93E360-8B95-ED11-AAD1-6045BD895CDC,Innovatie,Innovatiestrategie,OV-IN-Intake 2023,"haat - liefde verhouding , algemene sessies , ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243863,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-Ledenbezoek-Clevel-2019,"veel openstaande vacatures , roman 48 arbeider...",72F4DB06-1FF0-E911-8106-001DD8B72B61,347EE92A-C4BC-ED11-83FF-6045BD974EB2,Lidmaatschap,Retentie (Lidmaatschap),OV - Ledenbezoek - 2023,"stijgende kosten , sociale mediajob , centrum ..."
243864,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-Ledengesprek-2020,"moeilijke tijden , getroffen bedrijf , 12 m lo...",8C3B745F-2615-EB11-8

In [ ]:
# Get the activiteitcode on 1 row
df_activiteitcode = df[["crm_account_id", "crm_activiteitscode_naam"]]
df_activiteitcode = df_activiteitcode.pivot_table(
    index="crm_account_id", columns="crm_activiteitscode_naam", aggfunc="size"
).reset_index()
# All NaN values are replaced by 0
# All values bigger than 0 are replaced by 1
df_activiteitcode = df_activiteitcode.fillna(0)
# All numbered values are replaced by 
df_activiteitcode = df_activiteitcode.map(lambda x: 1 if not isinstance(x, str) and 0 < x  else x)
df_activiteitcode

crm_activiteitscode_naam,crm_account_id,Agrarische & bio-industrie,Automobiel- en Tweewielerindustrie,Bouw,"Chemie, petrochemie",Consultancy,"Distributie, logistiek en transport",Energie,Grafische industrie en diensten,Groothandel,...,Media,Milieu,Overige industrie & diensten,Papier & karton,Technologische industrie & diensten,Telecom & IT,"Textiel, kleding en confectie",Vastgoed,Voeding,Zorg
0,002C0FBC-C268-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,004E24A6-0A69-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,011C5162-D268-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,022C0FBC-C268-E111-B43A-00505680000A,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,023B965B-C268-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,FC9FD91C-C368-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,FD24E8A3-C268-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,FEEAB843-0969-E111-B43A-00505680000A,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202,FF24E8A3-C268-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Bring the activiteitcode back into the main dataframe
df = df.drop(columns=["crm_activiteitscode_naam"])
df = pd.merge(df, df_activiteitcode, left_on="crm_account_id", right_on="crm_account_id")
df

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,...,Media,Milieu,Overige industrie & diensten,Papier & karton,Technologische industrie & diensten,Telecom & IT,"Textiel, kleding en confectie",Vastgoed,Voeding,Zorg
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3806752A-D9E0-E711-80EE-001DD8B72B61,OV-P-Groep 268-PP.Toeleveranciers Retail-Plato...,2018-11-23 06:00:00,Project,Offline,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10732,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10733,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10734,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10735,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
def parse_keyphrases(
    df,
    anchor_name,
    column_name,
):
    # Create a dataframe with the keyphrases
    df_keyphrases = df[[anchor_name, column_name]].copy()

    # Get all distinct campagne_types
    campagne_types = df["crm_campagne_type_campagne"].unique()
    for campagne_type in campagne_types:
        # Create a new column for each campagne_type
        df_keyphrases.loc[:, anchor_name + "_" + campagne_type] = df_keyphrases.apply(
            lambda x: 1
            if isinstance(x[column_name], str) and (campagne_type.lower() in x[column_name].lower())
            else 0,
            axis=1,
        )

    # Drop the original column
    df_keyphrases = df_keyphrases.drop(columns=[column_name])
    # Join back into the main dataframe
    df = df.drop(columns=[anchor_name])
    df = pd.merge(df, df_keyphrases, left_index=True, right_index=True)
    return df


df = parse_keyphrases(
    df, "crm_afspraak_betreft_account_id", "crm_afspraak_betreft_account_keyphrases"
)
df = parse_keyphrases(
    df,
    "crm_afspraak_betreft_contact_id",
    "crm_afspraak_betreft_contactfiche_keyphrases",
)
df

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,...,crm_afspraak_betreft_account_id_Opleiding,crm_afspraak_betreft_account_id_Projectgebonden,crm_afspraak_betreft_account_id_Campagne,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contact_id_Infosessie,crm_afspraak_betreft_contact_id_Project,crm_afspraak_betreft_contact_id_Netwerkevenement,crm_afspraak_betreft_contact_id_Opleiding,crm_afspraak_betreft_contact_id_Projectgebonden,crm_afspraak_betreft_contact_id_Campagne
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,5B93E360-8B95-ED11-AAD1-6045BD895CDC,0,0,0,0,0,0
1,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,1,0,0,5B93E360-8B95-ED11-AAD1-6045BD895CDC,0,0,0,0,0,0
2,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,1,0,0,3146948E-6B40-EB11-8116-001DD8B72B61,0,1,0,1,0,0
3,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,3146948E-6B40-EB11-8116-001DD8B72B61,0,1,0,1,0,0
4,3806752A-D9E0-E711-80EE-001DD8B72B61,OV-P-Groep 268-PP.Toeleveranciers Retail-Plato...,2018-11-23 06:00:00,Project,Offline,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,5B93E360-8B95-ED11-AAD1-6045BD895CDC,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10732,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,F52E46AF-AFAE-EC11-9840-000D3A2788D3,0,1,0,0,0,0
10733,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,347EE92A-C4BC-ED11-83FF-6045BD974EB2,0,0,0,0,0,0
10734,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,347EE92A-C4BC-ED11-83FF-6045BD974EB2,0,0,0,0,0,0
10735,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,347EE92A-C4BC-ED11-83FF-6045BD974EB2,0,0,0,0,0,0


In [ ]:
df.describe()

,crm_campagne_startdatum,crm_financieledata_toegevoegde_waarde,crm_financieledata_fte,crm_financieledata_aantal_maanden,crm_financieledata_boekjaar,aantal_klachten,Agrarische & bio-industrie,Automobiel- en Tweewielerindustrie,Bouw,"Chemie, petrochemie",...,crm_afspraak_betreft_account_id_Netwerkevenement,crm_afspraak_betreft_account_id_Opleiding,crm_afspraak_betreft_account_id_Projectgebonden,crm_afspraak_betreft_account_id_Campagne,crm_afspraak_betreft_contact_id_Infosessie,crm_afspraak_betreft_contact_id_Project,crm_afspraak_betreft_contact_id_Netwerkevenement,crm_afspraak_betreft_contact_id_Opleiding,crm_afspraak_betreft_contact_id_Projectgebonden,crm_afspraak_betreft_contact_id_Campagne
count,10737,1.073700e+04,10737.000000,10737.000000,10737.000000,10737.000000,10737.000000,10737.000000,10737.000000,10737.000000,...,10737.0,10737.000000,10737.0,10737.000000,10737.000000,10737.000000,10737.000000,10737.00000,10737.0,10737.000000
mean,2020-09-17 19:57:45.716680704,2.522352e+07,221.205830,12.067710,2020.213188,6.442023,0.006333,0.034088,0.015833,0.089597,...,0.0,0.264227,0.0,0.020862,0.018627,0.163547,0.004284,0.19363,0.0,0.011176
min,2017-09-10 14:30:00,-1.459560e+05,0.000000,5.000000,2017.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000
25%,2019-06-19 16:00:00,1.684189e+06,17.000000,12.000000,2019.000000,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000
50%,2020-12-04 09:00:00,7.294452e+06,65.000000,12.000000,2020.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000
75%,2021-10-19 17:30:00,2.455244e+07,191.000000,12.000000,2021.000000,8.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000
max,2022-12-21 19:00:00,5.633480e+08,7606.000000,21.000000,2022.000000,24.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.00000,0.0,1.000000
std,NaN,7.029025e+07,725.120062,0.943237,1.361359,5.525481,0.079333,0.181463,0.124835,0.285616,...,0.0,0.440941,0.0,0.142930,0.135211,0.369881,0.065317,0.39516,0.0,0.105131


In [ ]:
# Get a random sample of the df_campaigns with legthg = len(df)

def upsample(df_):
    df = df_.copy()
    multiplier = int(len(df) / original_len) * 10

    df = pd.concat([df] * multiplier)
    df.reset_index(drop=True, inplace=True)
    return df

df_campaigns_sample = upsample(df_campaigns)
df_contacts_sample = upsample(df_contacts)


df_sample = pd.merge(
    df_campaigns_sample, df_contacts_sample, left_index=True, right_index=True
)
df_sample

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id_x,crm_contact_id_y,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,...,crm_afspraak_betreft_account_thema,crm_afspraak_betreft_account_subthema,crm_afspraak_betreft_account_onderwerp,crm_afspraak_betreft_account_keyphrases,crm_afspraak_betreft_account_id,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contactfiche_thema,crm_afspraak_betreft_contactfiche_subthema,crm_afspraak_betreft_contactfiche_onderwerp,crm_afspraak_betreft_contactfiche_keyphrases
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,6BE9AE3C-1B6D-E111-B43A-00505680000A,2E6C4D00-FEAD-ED11-AAD0-6045BD8956C9,HR Manager,0,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),Top Bronnen & Voka,"werner pycke [NAME] [NAME] , bedrijfsleider we...",195C408E-E4A0-EA11-8110-001DD8B72B61,8CB8DD89-16CE-ED11-B597-6045BD895554,Welt 2.0-2023,Intake,Welt 2.0 Intake,None
1,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,B61B99D1-34D0-EB11-8120-001DD8B72B61,2E6C4D00-FEAD-ED11-AAD0-6045BD8956C9,HR Manager,0,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),Top Bronnen & Voka,"werner pycke [NAME] [NAME] , bedrijfsleider we...",195C408E-E4A0-EA11-8110-001DD8B72B61,8CB8DD89-16CE-ED11-B597-6045BD895554,Welt 2.0-2023,Intake,Welt 2.0 Intake,None
2,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,A8A15926-6974-E111-B43A-00505680000A,2E6C4D00-FEAD-ED11-AAD0-6045BD8956C9,HR Manager,0,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),Top Bronnen & Voka,"werner pycke [NAME] [NAME] , bedrijfsleider we...",195C408E-E4A0-EA11-8110-001DD8B72B61,8CB8DD89-16CE-ED11-B597-6045BD895554,Welt 2.0-2023,Intake,Welt 2.0 Intake,None
3,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,48D9A655-5FD7-EC11-A7B5-000D3AACFF7A,2E6C4D00-FEAD-ED11-AAD0-6045BD8956C9,HR Manager,0,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),Top Bronnen & Voka,"werner pycke [NAME] [NAME] , bedrijfsleider we...",195C408E-E4A0-EA11-8110-001DD8B72B61,8CB8DD89-16CE-ED11-B597-6045BD895554,Welt 2.0-2023,Intake,Welt 2.0 Intake,None
4,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BD6E0D2C-446B-E111-B43A-00505680000A,2E6C4D00-FEAD-ED11-AAD0-6045BD8956C9,HR Manager,0,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),Top Bronnen & Voka,"werner pycke [NAME] [NAME] , bedrijfsleider we...",195C408E-E4A0-EA11-8110-001DD8B72B61,8CB8DD89-16CE-ED11-B597-6045BD895554,Welt 2.0-2023,Intake,Welt 2.0 Intake,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642495,6A4507DE-8FE3-E911-8106-001DD8B72B62,OV Conferentie - It's a smart world after all,2019-11-14 15:00:00,Netwerkevenement,Offline,40822672-7001-EA11-8107-001DD8B72B62,EE38AF1B-66FF-E711-80EF-001DD8B72B61,HR-officer,0,Nee,...,Welt 2.0,Dieptegesprek met Full Scan,Welt 2.0 Dieptegesprek met Full Scan,"celine desante , [NAME] [NAME] , strategische ...",81408337-2CF1-EB11-8125-001DD8B72B62,DAD0DBBD-9661-EC11-8F8F-000D3A2E70DE,Welt 2.0,Evaluatie actieplan,Welt 2.0 Geëvalueerd actieplan,None
642496,6A4507DE-8FE3-E911-8106-001DD8B72B62,OV Conferentie - It's a smart world after all,2019-11-14 15:00:00,Netwerkevenement,Offline,11501223-64EA-E911-8106-001DD8B72B62,F7A61266-84DD-E711-80ED-001DD8B72B62,Export Manager,0,Nee,...,Lidmaatschap,Retentie (Lidmaatschap),OV-klantenbezoek 2020,"legionella crisis , probleem waterzuivering , ...",A36BE5AB-8049-EA11-810B-001DD8B72B61,5BE63039-C115-EE11-8F6D-6045BD895D85,Internationaal Ondernemen,Marktinformatie,OV-C-Level-Internationalisatie,"[NAME] [NAME] [NAME] , veel ervaring corman , ..."
642497,6A4507DE-8FE3-E911-8106-001DD8B72B62,

In [ ]:
def parse_dataframe(df):
    # Filter out the rows where crm_boekjaar != YEAR(crm_campagne_startdatum)
    df["crm_campagne_startdatum"] = pd.to_datetime(df["crm_campagne_startdatum"])
    df["crm_campagne_startdatum_year"] = df["crm_campagne_startdatum"].dt.year

    df = df[df["crm_campagne_startdatum_year"] == df["crm_financieledata_boekjaar"]]
    df = df.drop(columns=["crm_campagne_startdatum_year"])
    # Get the activiteitcode on 1 row

    df_activiteitcode = df[["crm_account_id", "crm_activiteitscode_naam"]]
    df_activiteitcode = df_activiteitcode.pivot_table(
        index="crm_account_id", columns="crm_activiteitscode_naam", aggfunc="size"
    ).reset_index()
    # All NaN values are replaced by 0
    # All values bigger than 0 are replaced by 1
    df_activiteitcode = df_activiteitcode.fillna(0)
    # All numbered values are replaced by
    df_activiteitcode = df_activiteitcode.map(
        lambda x: 1 if not isinstance(x, str) and 0 < x else x
    )
    # Bring the activiteitcode back into the main dataframe


    df = df.drop(columns=["crm_activiteitscode_naam"])
    df = pd.merge(
        df, df_activiteitcode, left_on="crm_account_id", right_on="crm_account_id"
    )
    df = parse_keyphrases(
        df, "crm_afspraak_betreft_account_id", "crm_afspraak_betreft_account_keyphrases"
    )
    df = parse_keyphrases(
        df,
        "crm_afspraak_betreft_contact_id",
        "crm_afspraak_betreft_contactfiche_keyphrases",
    )
    return df

df_sample = parse_dataframe(df_sample)
df_o = parse_dataframe(df_o)

In [ ]:
df_o

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,...,crm_afspraak_betreft_account_id_Opleiding,crm_afspraak_betreft_account_id_Projectgebonden,crm_afspraak_betreft_account_id_Campagne,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contact_id_Infosessie,crm_afspraak_betreft_contact_id_Project,crm_afspraak_betreft_contact_id_Netwerkevenement,crm_afspraak_betreft_contact_id_Opleiding,crm_afspraak_betreft_contact_id_Projectgebonden,crm_afspraak_betreft_contact_id_Campagne
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,5B93E360-8B95-ED11-AAD1-6045BD895CDC,0,0,0,0,0,0
1,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,1,0,0,5B93E360-8B95-ED11-AAD1-6045BD895CDC,0,0,0,0,0,0
2,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,1,0,0,3146948E-6B40-EB11-8116-001DD8B72B61,0,1,0,1,0,0
3,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,3146948E-6B40-EB11-8116-001DD8B72B61,0,1,0,1,0,0
4,3806752A-D9E0-E711-80EE-001DD8B72B61,OV-P-Groep 268-PP.Toeleveranciers Retail-Plato...,2018-11-23 06:00:00,Project,Offline,BB38CFB9-D671-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,5B93E360-8B95-ED11-AAD1-6045BD895CDC,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10732,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,F52E46AF-AFAE-EC11-9840-000D3A2788D3,0,1,0,0,0,0
10733,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,347EE92A-C4BC-ED11-83FF-6045BD974EB2,0,0,0,0,0,0
10734,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,347EE92A-C4BC-ED11-83FF-6045BD974EB2,0,0,0,0,0,0
10735,A5B3FA54-EC46-EC11-8C62-6045BD8D2834,OV-IN-Subsidie- en financieringszitdag VL-WAL-FR,2022-01-18 09:00:00,Netwerkevenement,Online,34F81AE5-A66A-E111-B43A-00505680000A,Bedrijfsleider,0,Nee,Nee,...,0,0,0,347EE92A-C4BC-ED11-83FF-6045BD974EB2,0,0,0,0,0,0


In [ ]:
df_sample

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id_x,crm_contact_id_y,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,...,crm_afspraak_betreft_account_id_Infosessie,crm_afspraak_betreft_account_id_Campagne,crm_afspraak_betreft_account_id_Projectgebonden,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contact_id_Opleiding,crm_afspraak_betreft_contact_id_Netwerkevenement,crm_afspraak_betreft_contact_id_Project,crm_afspraak_betreft_contact_id_Infosessie,crm_afspraak_betreft_contact_id_Campagne,crm_afspraak_betreft_contact_id_Projectgebonden
0,1B445E22-A9CB-EC11-A7B5-000D3A20A90F,OV-Infosessie - Bedrijfsoverdracht: Wat met de...,2022-06-24 09:00:00,Opleiding,Offline,7EB669AA-8229-E211-B2FD-005056B06EB4,043D5861-75C3-E611-80E3-001DD8B72B62,Zaakvoerder,0,Nee,...,0,0,0,8FB29AE1-6E03-EE11-8F6E-6045BD895233,0,0,0,0,0,0
1,C6FAD0EC-63AF-EC11-9840-000D3A2788D3,OV-NW-Rode Loper Selectie-Ros Beiaard Ommegang...,2022-05-29 11:00:00,Netwerkevenement,Offline,C15A0D02-BCC6-EC11-A7B6-000D3A497120,043D5861-75C3-E611-80E3-001DD8B72B62,Zaakvoerder,0,Nee,...,0,0,0,8FB29AE1-6E03-EE11-8F6E-6045BD895233,0,0,0,0,0,0
2,C6FAD0EC-63AF-EC11-9840-000D3A2788D3,OV-NW-Rode Loper Selectie-Ros Beiaard Ommegang...,2022-05-29 11:00:00,Netwerkevenement,Offline,D9522C2D-F82D-E911-80FB-001DD8B72B62,043D5861-75C3-E611-80E3-001DD8B72B62,Zaakvoerder,0,Nee,...,0,0,0,8FB29AE1-6E03-EE11-8F6E-6045BD895233,0,0,0,0,0,0
3,7959C8D4-5D17-EA11-8109-001DD8B72B62,OV-NW-Infovergadering Wijnveld Zele,2020-01-08 18:00:00,Netwerkevenement,Offline,AAFFB4DC-6D12-E911-80FA-001DD8B72B61,043D5861-75C3-E611-80E3-001DD8B72B62,Zaakvoerder,0,Nee,...,0,0,0,8FB29AE1-6E03-EE11-8F6E-6045BD895233,0,0,0,0,0,0
4,302D81AA-7222-EA11-8109-001DD8B72B62,OV-Plato Academy 2020 - Kick-off workshop ALL,2020-02-18 09:00:00,Project,Offline,FF8DD2E3-FA1E-E911-80FB-001DD8B72B62,043D5861-75C3-E611-80E3-001DD8B72B62,Zaakvoerder,0,Nee,...,0,0,0,8FB29AE1-6E03-EE11-8F6E-6045BD895233,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28022,9C1C2E9D-7419-E911-80FA-001DD8B72B62,OV-IO-China techtrip: pré-event,2019-01-31 18:30:00,Netwerkevenement,Offline,BB83578F-227C-E211-A980-005056B06EB4,D31E8E91-7C93-EB11-811E-001DD8B72B62,"HR-medewerker Welzijn, ontwikkeling en leiders...",0,Nee,...,0,0,0,1F0F16BA-DEFE-ED11-8F6E-6045BD895B5A,0,0,0,0,0,0
28023,3F1607CF-65FC-E711-80EF-001DD8B72B61,OV-A&O-Focusgroep arbeidsmarktbevraging Haven,2018-01-25 06:00:00,Projectgebonden,Offline,71AB3B6C-68FC-E711-80EF-001DD8B72B61,D31E8E91-7C93-EB11-811E-001DD8B72B62,"HR-medewerker Welzijn, ontwikkeling en leiders...",0,Nee,...,0,0,0,E65D3E07-17CE-ED11-B597-6045BD895554,0,0,0,0,0,0
28024,F0B9663C-55FF-E711-80EF-001DD8B72B61,OV-NW-JO-Tinder Invest voorjaar 2018,2018-03-22 06:00:00,Netwerkevenement,Offline,FDD10500-D96F-E111-B43A-00505680000A,D31E8E91-7C93-EB11-811E-001DD8B72B62,"HR-medewerker Welzijn, ontwikkeling en leiders...",0,Nee,...,0,0,0,3219EF83-C5FE-ED11-8F6E-6045BD8956C9,0,0,0,0,0,0
28025,F0B9663C-55FF-E711-80EF-001DD8B72B61,OV-NW-JO-Tinder Invest voorjaar 2018,2018-03-22 06:00:00,Netwerkevenement,Offline,0FF180CE-A1A4-E511-A0DC-005056B06EC4,9814D611-5821-E511-93F3-005056B06EC4,Algemeen Directeur,0,Nee,...,0,0,0,98B27A37-EA33-EC11-8124-001DD8B72B61,0,0,1,0,0,0


In [ ]:
min_length = min(len(df_o), len(df_sample))
df_o = df_o.sample(min_length)
df_sample = df_sample.sample(min_length)

df_o["attended"] = 1
df_sample["attended"] = 0

df = pd.concat([df_o, df_sample])
df

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,...,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contact_id_Infosessie,crm_afspraak_betreft_contact_id_Project,crm_afspraak_betreft_contact_id_Netwerkevenement,crm_afspraak_betreft_contact_id_Opleiding,crm_afspraak_betreft_contact_id_Projectgebonden,crm_afspraak_betreft_contact_id_Campagne,attended,crm_contact_id_x,crm_contact_id_y
7600,CA56CAFB-4E35-EB11-8116-001DD8B72B61,OV-Stad Aalst- LinkedIn,2020-12-16 12:00:00,Opleiding,Online,7087F42E-1873-E111-B43A-00505680000A,Zaakvoerder,0,Nee,Nee,...,508B4165-28D1-EC11-A7B5-000D3A4800DE,0,0,0,0,0,0,1,NaN,NaN
907,598E4DA4-CF59-E911-80FD-001DD8B72B62,OV-NW-VIP-GentJazz-JamieCullum-Juni,2019-06-30 18:00:00,Netwerkevenement,Offline,9A7F9DFD-3F6E-E111-B43A-00505680000A,Zaakv. vennootsch. beperkte aansprakel.,0,Nee,Nee,...,726449A0-FDFF-EB11-8125-001DD8B72B62,0,0,0,0,0,0,1,NaN,NaN
10087,3895CCCB-D32D-E911-80FB-001DD8B72B62,OV-NW-L&L: Verleiden met Philip Van Cauteren e...,2019-04-24 18:00:00,Netwerkevenement,Offline,1C34BE2F-6CE6-E811-80F8-001DD8B72B61,Managing Director IDS EMEA,0,Nee,Nee,...,02AE5BCA-B903-ED11-82E5-000D3A2A9927,0,0,0,0,0,0,1,NaN,NaN
9566,147B8346-8D44-ED11-BBA2-6045BD8956C9,OV-welt-TT-Slotevent december 2022,2022-12-16 09:00:00,Project,Offline,3C044E7A-B566-E511-895A-005056B06EC4,Recruitment & Training Officer,0,Nee,Nee,...,46F1C451-757F-EC11-8D21-6045BD8E001C,0,0,0,0,0,0,1,NaN,NaN
2259,21624DC0-695D-EA11-810D-001DD8B72B61,OV-NW- Voka Politica Oudenaarde 2020,2020-10-30 08:00:00,Netwerkevenement,Offline,C04A3879-9E6D-E111-B43A-00505680000A,Zaakvoerder,0,Nee,Ja,...,E37F6D7E-B9E2-ED11-A7C7-6045BD895554,0,0,0,1,0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6474,D18E5021-B135-E911-80FB-001DD8B72B62,OV-Innovatiecafé,2019-04-02 16:00:00,Opleiding,Offline,NaN,Zaakvoerder,0,Ja,Nee,...,33F51F29-D29B-EC11-B400-002248800905,0,0,0,0,0,0,0,C39298DA-D6E4-E411-AC4D-005056B06EC4,8EC46C42-8BA6-E611-80DE-001DD8B72B61
12359,4033D485-24B6-EA11-8111-001DD8B72B61,OV-NW-Mobiliteitsdag 2020,2020-09-21 09:00:00,Netwerkevenement,Offline,NaN,Directeur,0,Nee,Nee,...,4565C597-76E2-EC11-BB3C-000D3ABD8B95,0,0,0,0,0,0,0,76B007CA-E175-E211-A980-005056B06EB4,D4E05EE6-8072-E111-B43A-00505680000A
596,F2F021B0-F328-E811-80F0-001DD8B72B61,OV-Inspiratie event - Guy Van Wymeersch getuig...,2018-05-02 06:00:00,Opleiding,Offline,NaN,Verantwoordelijke Productie,0,Ja,Ja,...,25B29386-BC2A-ED11-9DB1-0022488013C6,0,0,0,1,0,0,0,60F1FC30-8431-E811-80F0-001DD8B72B61,9A3A914D-BC6B-E111-B43A-00505680000A
9571,CFF2BA02-E99C-EB11-811E-001DD8B72B62,OV-NW-210531-VokaUpdate-QuickRefresh 1-Voorjaa...,2021-05-31 11:00:00,Netwerkevenement,Online,NaN,HR Professional,0,Nee,Nee,...,400FE4F9-CCFE-ED11-8F6E-6045BD8956C9,0,0,0,0,0,0,0,FDAE2E28-3547-EA11-810C-001DD8B72B62,C44641BE-4324-ED11-B83D-000D3A2D2E2A


In [ ]:
df.to_csv("data/data.csv", index=False)